# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_weather = "Output Data/cities_weather_data.csv"
weather_data = pd.read_csv(city_weather)
weather_data = weather_data.rename(columns = {"Unnamed: 0": "Record Number"})

weather_data.head()

,Record Number,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Puerto Ayora,-0.74,-90.35,19.44,84,54,2.24,EC,1600850598
1,1,Bluff,-46.60,168.33,12.22,77,44,2.24,NZ,1600850404
2,2,Kresek,-6.13,106.38,35.00,66,20,4.10,ID,1600850599
3,3,Port Alfred,-33.59,26.89,20.54,51,31,2.45,ZA,1600850599
4,4,Gornopravdinsk,60.05,69.90,4.09,95,100,5.26,RU,1600850599


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key = g_key)
city_location = weather_data[["Lat", "Lng"]].astype(float)
city_humidity = weather_data["Humidity"].astype(float)

In [5]:
figs = gmaps.figure()

heat_layer = gmaps.heatmap_layer(city_location, weights = city_humidity, dissipating=False, max_intensity=100, point_radius = 1)
figs.add_layer(heat_layer)

figs

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cloud_df = weather_data.loc[weather_data["Cloudiness"] == 0,:]
wind_speed_df = cloud_df.loc[cloud_df["Wind Speed"] < 20,:]
perfect_weather_df = wind_speed_df.loc[wind_speed_df["Max Temp"] < 80, :]

perfect_weather_df.head()

,Record Number,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,Saskylakh,71.92,114.08,10.67,48,0,7.46,RU,1600850601
25,25,Ozinki,51.18,49.68,20.54,45,0,7.67,RU,1600850602
35,35,Kasempa,-13.46,25.83,28.58,27,0,4.97,ZM,1600850603
36,36,Yulara,-25.24,130.99,27.00,13,0,4.60,AU,1600850603
38,38,Khatanga,71.97,102.50,12.51,50,0,5.77,RU,1600850604


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df = perfect_weather_df
hotel_df["Hotel Name"] = ""

hotel_df.head()

,Record Number,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,19,Saskylakh,71.92,114.08,10.67,48,0,7.46,RU,1600850601,
25,25,Ozinki,51.18,49.68,20.54,45,0,7.67,RU,1600850602,
35,35,Kasempa,-13.46,25.83,28.58,27,0,4.97,ZM,1600850603,
36,36,Yulara,-25.24,130.99,27.00,13,0,4.60,AU,1600850603,
38,38,Khatanga,71.97,102.50,12.51,50,0,5.77,RU,1600850604,


In [8]:
hotel_params = {"radius": 5000, "types": "lodging", "keyword": "Hotel", "key": g_key}

print("Finding the nearest hotel")
for index, row in hotel_df.iterrows():
    hotel_lat = row["Lat"]
    hotel_lng = row["Lng"]
    hotel_params["location"] = f"{hotel_lat},{hotel_lng}"
    
    hotel_find_data = requests.get(base_url, params = hotel_params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_find_data["results"][0]["name"]
        print("Hotel found")
    except(IndexError, KeyError):
        print("Hotel not found")

hotel_df.head()

Finding the nearest hotel
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found
Hotel found
Hotel found
Hotel not found
Hotel found
Ho

,Record Number,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,19,Saskylakh,71.92,114.08,10.67,48,0,7.46,RU,1600850601,
25,25,Ozinki,51.18,49.68,20.54,45,0,7.67,RU,1600850602,at Rest
35,35,Kasempa,-13.46,25.83,28.58,27,0,4.97,ZM,1600850603,Comfort Stay Lodge
36,36,Yulara,-25.24,130.99,27.00,13,0,4.60,AU,1600850603,Sails in the Desert
38,38,Khatanga,71.97,102.50,12.51,50,0,5.77,RU,1600850604,"Gostinitsa ""Zapolyar'ye"""


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

hotel_layer = gmaps.marker_layer(locations2, hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))